# 5.4 自定义层

## 5.4.1 不带参数的层

In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [9]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))
layer

CenteredLayer()

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
net

Sequential(
  (0): Linear(in_features=8, out_features=128, bias=True)
  (1): CenteredLayer()
)

In [7]:
Y = net(torch.rand(4, 8))
Y.mean(),Y.shape

(tensor(7.4506e-09, grad_fn=<MeanBackward0>), torch.Size([4, 128]))

## 5.4.2 带参数的层

In [10]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [12]:
linear = MyLinear(5, 3)
linear.weight,linear

(Parameter containing:
 tensor([[-0.1897, -0.3892,  0.7888],
         [ 0.0935, -0.3352, -1.2162],
         [-0.0937,  0.4617,  0.4041],
         [-2.1763, -0.3229, -1.1441],
         [ 1.5124, -0.5595, -1.7637]], requires_grad=True),
 MyLinear())

In [13]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.0148, 0.0000],
        [2.1973, 0.5210, 0.0000]])

In [15]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

## 5.4.3 小结
* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与库中的任何现有层不同。
* 在自定义层定义完成后，就可以在任意环境和网络结构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。